## Set up for Google Colab 

In [ ]:
from io import StringIO
from scipy.io import mmread
from scipy.io import mminfo
from google.colab import drive
drive.mount("/content/drive")
datAddr = "/content/drive/MyDrive/CMSC818J_Test/"
from scipy.sparse import csr_matrix
from scipy import sparse
import numpy as np
import sys
import os
import csv
import math

Mounted at /content/drive


## Optimized Architecture 
 

## Assumptions
1. Each multiplication and accumulation takes 3 cycles
2. Reading the rows from memory takes 2 cycles
3. Storing the result of multiplication and accumulation takes 1 cycle. NO merging required since we are implementing inner product. 
4. PEs write back the output in parallel and can read the input in parallel
5. We will sort 8 rows at a time as our assumption is that given our bandwidth, we can read 8 rows at a time

In [ ]:
# ENUMS for PE_stats
BUSY_CYCLES = 1 
MAX_NUM_NON_ZEROS = 2
MIN_NUM_NON_ZEROS = 3
IDLE_CYCLES = 4
FIFO_BUFFER_LENGTH = 8

CYCLES_TO_READ = 8
CYCLES_TO_SORT = 6
CYCLES_TO_PUT_BUFFER = 1 # per element
CYCLES_TO_MUL_AND_ACCUMULATE = 6 # per element multiplication
WRITE_BACK_RATE = 1000 # can write 1000 elements at a time 

In [ ]:
# Adds the same number of idle cycles to each of PEs
def add_idle_cycles (cycles_num):
  for i in range(total_PEs):
    PE_stats[i][IDLE_CYCLES] += cycles_num 

In [ ]:
# Adds the different number of idle cycles to each of PEs by subtracting the 
# max_non_per_PE which represents the cycles for the slowest PEs from the busy 
# cycles of the other PEs once after all the allocations are finished. 
def add_idle_cycles2(idle_cycle_list, max_nonzero_per_PE, PE_stats):
  for i in range(total_PEs):
    PE_stats[i][IDLE_CYCLES] += max_nonzero_per_PE - idle_cycle_list[i] 
  return PE_stats

In [ ]:
# Checks whether the busy and idle cycles add up to the total number of 
# cycles for each of the output files.  
def check_cycle_accuracy(filename):
  for pe in PE_stats:
    if pe[BUSY_CYCLES] + pe[IDLE_CYCLES] != TOTAL_NUMBER_OF_CYCLES:
      print(filename + " Doesn't match up" + str(total_PEs))

In [ ]:
# Simulates PE MAC operations
def PE_mult(output_col_num, PE_num):
  total_cycles_spent = 0 
  PE_to_process = PE_lst.pop(0)
  row_num = PE_output_index.pop(0) 
  accumulation = 0
  # optimization for inner product: whichever dictionary has smallest non-zeros, we iterate over that. 
  if len(B_dict) < len(PE_to_process):
    for col,data in B_dict.items():
      if col in PE_to_process:
        # multiply and then accumulate  
        accumulation += PE_to_process[col] * data 
        PE_stats[PE_num][BUSY_CYCLES] += CYCLES_TO_MUL_AND_ACCUMULATE
        total_cycles_spent += 1 
    output_mat2[row_num][output_col_num] = accumulation 
  else:
    for col,data in PE_to_process.items():
      if col in B_dict:
        # multiply and then accumulate  
        accumulation += B_dict[col] * data 
        PE_stats[PE_num][BUSY_CYCLES] += CYCLES_TO_MUL_AND_ACCUMULATE
        total_cycles_spent += 1 
    output_mat2[row_num][output_col_num] = accumulation 
  # Calculate max number of cycles  
  if len(PE_to_process) > PE_stats[PE_num][MAX_NUM_NON_ZEROS]: 
      PE_stats[PE_num][MAX_NUM_NON_ZEROS] = len(PE_to_process)
  elif len(PE_to_process) < PE_stats[PE_num][MIN_NUM_NON_ZEROS]: 
      PE_stats[PE_num][MIN_NUM_NON_ZEROS] = len(PE_to_process)
    
  return total_cycles_spent   



In [ ]:
 # Read the rows from matrices and add them to FIFO buffer
def sort_computer_rows(data_index_A, col_index_A, pointer_index_A, output_row_index):
  PE_rows_read = dict() 
  PE_rows_index = dict()
  max_nonzero_per_PE = 0
  cycles_to_sort = 0
  idle_cycle_list = [] # keeps track of idle cycles for each PE per allocation
  # Because of FIFO buffer length, there are lower number of cycles
  for i in range(FIFO_BUFFER_LENGTH):
    if pointer_index_A < mat_csrA_len:
      numOfElems = (mat_csr.indptr[pointer_index_A + 1] - mat_csr.indptr[pointer_index_A])
      data_row_A = mat_csr.data[data_index_A:numOfElems+data_index_A]
      row_A = mat_csr.indices[col_index_A:numOfElems+data_index_A]
      if numOfElems in PE_rows_read: 
        PE_rows_read[numOfElems].append(dict(zip(row_A, data_row_A)))
        PE_rows_index[numOfElems].append(output_row_index)
        data_index_A += numOfElems
        col_index_A += numOfElems
        pointer_index_A += 1
        output_row_index += 1
      else:
        PE_rows_read[numOfElems] = []
        PE_rows_read[numOfElems].append(dict(zip(row_A, data_row_A)))
        PE_rows_index[numOfElems] = []
        PE_rows_index[numOfElems].append(output_row_index)
        data_index_A += numOfElems
        col_index_A += numOfElems
        pointer_index_A += 1
        output_row_index += 1 
    else:
      break

  # sort the rows in the buffer and then add then to the FIFO which is PE_lst
  non_zero_keys = list(PE_rows_read.keys())
  non_zero_keys.sort(reverse = True)
  cycles_to_sort += CYCLES_TO_SORT
  
  for k in non_zero_keys:
    for PE_rows in PE_rows_read[k]:
      PE_lst.append(PE_rows)
      PE_output_index.append(PE_rows_index[k].pop(0)) # keeps track of row num after sorting
    cycles_to_sort += CYCLES_TO_PUT_BUFFER
  
  return (data_index_A, col_index_A, pointer_index_A, output_row_index, cycles_to_sort)





In [ ]:
# Read the matrices and multiply them by each other 
directory = os.fsencode(datAddr)
for pe_num in range(2, 10, 2):
  with open('/content/drive/MyDrive/CMSC818J_Data/Optimized_PE' + str(pe_num) + '_16fifo.csv', 'w', newline='') as file:
    writer = csv.writer(file)   
    for file in os.listdir(directory):
      filename = os.fsdecode(file)
      print(filename)

      ## Read the file 
      m = mmread(datAddr+filename)
      mat_csr = m.tocsr() # convert the first matrix to csr 
      mat_csrB = m.tocsc() # convert the first matrix to csc

      total_PEs= pe_num # number of PEs available. Can change this number as required
      PE_stats = [] # Keeps track of the metrics

      for i in range(total_PEs):
        PE_stat = dict()
        PE_stat[BUSY_CYCLES] = 0 
        PE_stat[MAX_NUM_NON_ZEROS] = 0 
        PE_stat[MIN_NUM_NON_ZEROS ] = sys.maxsize
        PE_stat[IDLE_CYCLES] = 0 
        PE_stats.append(PE_stat)
        PE_stats
      

      max_total_cycles = 0
      mat_csrB_len = len(mat_csrB.indptr) - 1
      mat_csrA_len = len(mat_csr.indptr) - 1
      output_mat_len = len(mat_csrB.indptr) - 1
      PE_num = 0
      # acts as the buffer that stores rows to be allocated to PEs
      PE_lst = []
      PE_output_index = []
      # matrix to store the file output 
      output_mat2 = [[0 for _ in range(output_mat_len)] for _ in range(output_mat_len)]
      data_index_B = 0 
      row_index_B = 0
      pointer_index_B = 0 
      output_col_index = 0
      output_row_index = 0
      cycles_to_sort = 0

      # Total number of cycles is whichever PE took the longest 
      TOTAL_NUMBER_OF_CYCLES = 0 

      TOTAL_NUMBER_OF_CYCLES += CYCLES_TO_READ # For reading elements, one time cost rest of them are amortized with PE computations
      add_idle_cycles(CYCLES_TO_READ)


      for j in range(mat_csrB_len): 
        # Decode col of matrix B from csc format
        numOfElems_B = (mat_csrB.indptr[pointer_index_B + 1] - mat_csrB.indptr[pointer_index_B])
        data_row_B = mat_csrB.data[data_index_B:numOfElems_B+data_index_B]
        col_B = mat_csrB.indices[row_index_B:numOfElems_B+data_index_B]

        # The decoded cols and corresponding data values are zipped into a dictionary 
        # data structure to easily pattern match with rows when computing inner product
        B_dict = dict(zip(col_B, data_row_B))
        data_index_B += numOfElems_B
        row_index_B += numOfElems_B
        pointer_index_B += 1
        data_index_A = 0 # these values need to updated in if not PE
        col_index_A = 0
        pointer_index_A = 0
        output_row_index = 0
        for i in range(mat_csrA_len):
          # Decode rows of matrix A from csr format 
          if not PE_lst:
            data_index_A, col_index_A, pointer_index_A, output_row_index, cycles_to_sort = sort_computer_rows(data_index_A, col_index_A, pointer_index_A, output_row_index)
            # allocate it to a PE using round-robin strategy 
          total_computation_cycles = PE_mult(output_col_index, PE_num)
          PE_stats[PE_num][IDLE_CYCLES] += math.ceil(total_computation_cycles/WRITE_BACK_RATE)
          PE_num = (PE_num + 1) % total_PEs 
        output_col_index += 1 

      # Keeps track of max total number of cycles and once all the allocations are 
      # finished the idle cycles are added. The PEs that had the longest workload 
      # is used for calculating idle cycles for other PEs.  
      max_cycles = []
      for pe in range(pe_num):
        max_cycles.append(PE_stats[pe][BUSY_CYCLES])
        max_total_cycles = max(PE_stats[pe][BUSY_CYCLES] + PE_stats[pe][IDLE_CYCLES], max_total_cycles)
      TOTAL_NUMBER_OF_CYCLES += max_total_cycles
      max_num = max(max_cycles)

      string = [filename, TOTAL_NUMBER_OF_CYCLES]
      for pe in range(pe_num):
        for stat_num in range(1, 5, 1):
          if stat_num == IDLE_CYCLES:
            # assumption is that the rest of the PEs will wait for the main PE
            # The add_idle_cycles2 computes the number of cycles 
            # each PE has to wait for the slowest PE to finish computation. 
            idle_cycles_to_add = max_num - PE_stats[pe][BUSY_CYCLES]
            string.append(PE_stats[pe][stat_num] + idle_cycles_to_add)
          else:
            string.append(PE_stats[pe][stat_num])
      output_mat2_reshape = np.reshape(output_mat2, (output_mat_len, output_mat_len))
      correct_output = np.dot(m.todense(),m.todense())
      # Validates if matrix multiplication is done correctly
      print(np.allclose(output_mat2_reshape, correct_output))
      # write back to the output file 
      writer.writerow(string)



In [ ]:
# Read the matrices and multiply them by a randomly generated matrix
directory = os.fsencode(datAddr)

for pe_num in range(2, 10, 2):
  with open('/content/drive/MyDrive/CMSC818J_Data/Optimized_PEAB' + str(pe_num) + '_16fifo.csv', 'w', newline='') as file:
    writer = csv.writer(file)   
    for file in os.listdir(directory):
      filename = os.fsdecode(file)
      print(filename)

      ## Read the file 
      m = mmread(datAddr+filename)
      m2 = mmread("/content/drive/MyDrive/psmigr_3.mtx") # serves as the random matrix 
      mat_csr = m.tocsr() # convert the first matrix to csr 
      mat_csrB = m2.tocsc() # convert the second matrix to csc 
      mat_csrB.resize(mat_csr.shape) # reshape so that it can be multiplied to first matrix

      total_PEs= pe_num # number of PEs available. Can change this number as required
      PE_stats = [] # to store the metrics

      for i in range(total_PEs):
        PE_stat = dict()
        PE_stat[BUSY_CYCLES] = 0 
        PE_stat[MAX_NUM_NON_ZEROS] = 0 
        PE_stat[MIN_NUM_NON_ZEROS ] = sys.maxsize
        PE_stat[IDLE_CYCLES] = 0 
        PE_stats.append(PE_stat)
        PE_stats
      

      max_total_cycles = 0
      mat_csrB_len = len(mat_csrB.indptr) - 1
      mat_csrA_len = len(mat_csr.indptr) - 1
      # output matrix size 
      output_mat_lenX = len(mat_csr.indptr) - 1
      output_mat_lenY = len(mat_csrB.indptr) - 1
      PE_num = 0
      # acts as the buffer that stores rows to be allocated to PEs
      PE_lst = []
      PE_output_index = []
      # matrix to store the file output 
      output_mat2 = np.zeros((output_mat_lenX, output_mat_lenY))
      data_index_B = 0 
      row_index_B = 0
      pointer_index_B = 0 
      output_col_index = 0
      output_row_index = 0
      cycles_to_sort = 0

      # Total number of cycles is whichever PE took the longest 
      TOTAL_NUMBER_OF_CYCLES = 0 

      TOTAL_NUMBER_OF_CYCLES += CYCLES_TO_READ # For reading elements, one time cost rest of them are amortized with PE computations
      add_idle_cycles(CYCLES_TO_READ)


      for j in range(mat_csrB_len): 
        # Decode col of matrix B from csc format
        numOfElems_B = (mat_csrB.indptr[pointer_index_B + 1] - mat_csrB.indptr[pointer_index_B])
        data_row_B = mat_csrB.data[data_index_B:numOfElems_B+data_index_B]
        col_B = mat_csrB.indices[row_index_B:numOfElems_B+data_index_B]

        # The decoded cols and corresponding data values are zipped into a dictionary 
        # data structure to easily pattern match with rows when computing inner product
        B_dict = dict(zip(col_B, data_row_B))
        data_index_B += numOfElems_B
        row_index_B += numOfElems_B
        pointer_index_B += 1
        data_index_A = 0 # these values need to updated in if not PE
        col_index_A = 0
        pointer_index_A = 0
        output_row_index = 0
        for i in range(mat_csrA_len):
          # Decode rows of matrix A from csr format 
          if not PE_lst:
            data_index_A, col_index_A, pointer_index_A, output_row_index, cycles_to_sort = sort_computer_rows(data_index_A, col_index_A, pointer_index_A, output_row_index)
            # allocate it to a PE using round-robin strategy 
          total_computation_cycles = PE_mult(output_col_index, PE_num)
          PE_stats[PE_num][IDLE_CYCLES] += math.ceil(total_computation_cycles/WRITE_BACK_RATE)
          PE_num = (PE_num + 1) % total_PEs 
        output_col_index += 1 
      
      # Keeps track of max total number of cycles and once all the allocations are 
      # finished the idle cycles are added. The PEs that had the longest workload 
      # is used for calculating idle cycles for other PEs.  
      max_cycles = []
      for pe in range(pe_num):
        max_cycles.append(PE_stats[pe][BUSY_CYCLES])
        max_total_cycles = max(PE_stats[pe][BUSY_CYCLES] + PE_stats[pe][IDLE_CYCLES], max_total_cycles)
      TOTAL_NUMBER_OF_CYCLES += max_total_cycles
      max_num = max(max_cycles)

      string = [filename, TOTAL_NUMBER_OF_CYCLES]
      for pe in range(pe_num):
        for stat_num in range(1, 5, 1):
          if stat_num == IDLE_CYCLES:
            # assumption is that the rest of the PEs will wait for the main PE
            # The add_idle_cycles2 computes the number of cycles 
            # each PE has to wait for the slowest PE to finish computation
            idle_cycles_to_add = max_num - PE_stats[pe][BUSY_CYCLES]
            string.append(PE_stats[pe][stat_num] + idle_cycles_to_add)
          else:
            string.append(PE_stats[pe][stat_num])
      output_mat2_reshape = np.reshape(output_mat2, (output_mat_lenX, output_mat_lenY))
      correct_output = (mat_csr * mat_csrB).todense()   
      # Validates if matrix multiplication is done correctly
      print(np.allclose(output_mat2_reshape, correct_output))
      writer.writerow(string)